In [1]:
# import necessary libraries

import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import all datasources and convert them into pandas dataframes

path = '/Users/ilaydabekircan/Documents/Vision Bridge/DS Assignment/AssignmentData'

csv_files = [file for file in os.listdir(path) if file.endswith('.csv')]

for file in csv_files:
    df_name = file.split('.')[0]
    globals()[df_name] = pd.read_csv(os.path.join(path, file))

In [3]:
# print names of each dataframe
df_names = [name for name, obj in globals().items() if isinstance(obj, pd.DataFrame)]
df_names

['Customers',
 'ChannelsCampaigns',
 'Products',
 'StoresCampaigns',
 'Orders',
 'OrderItems',
 'Users',
 'AudiencesCampaigns',
 'Retailers',
 'MarketingCampaigns',
 'CampaignResults',
 'Tracking',
 'Channels',
 'Audiences',
 'Transactions',
 'BehavioralData',
 'Stores']

# BUYERS

In [5]:
print('Number of customers in our dataset', Customers['CustomerID'].nunique())
print('Number of customers who bought something:', Orders['CustomerID'].nunique())
print('Number of customers who didnt buy something:', Customers['CustomerID'].nunique() - Orders['CustomerID'].nunique())

Number of customers in our dataset 5000
Number of customers who bought something: 917
Number of customers who didnt buy something: 4083


Although most of the customers didn't buy anything, we can still learn from their behaviours.

First, we will merge only the datasets with customer and product information into CustomerBuys and the store information will be gathered under SellerSells df. In CustomerBuys, Customers df will be the main dataset and therefore left join will be used while merging other datasets into Customers.

In [8]:
# function to define season by month
def find_season(month_num):
    if month_num in [12, 1, 2]:
        return 1
    elif month_num in [3, 4, 5]:
        return 2
    elif month_num in [6, 7, 8]:
        return 3
    elif month_num in [9, 10, 11]:
        return 4

Customers df has only two columns with missing information.

In [10]:
customers_na = Customers.isna().sum()
customers_na[customers_na > 0]

Occupation    1388
Industry      1388
dtype: int64

Orders df has no missing information within itself.

In [12]:
orders_na = Orders.isna().sum()
orders_na[orders_na > 0]

Series([], dtype: int64)

Before merging Customers and Orders, both dataframes need preprocessing. Some columns in Customers are renamed to prevent confusion in later steps. Also, missing information in occupation and industry columns are labeled as 'Unknown' while the unnecessary columns are dropped. Email, phone and the data creation is unnecessary for customer segmentation purposes. Since we have CustomerID, name is also unnecessary. Country is not used in the dataset since all observations are from the US.

By using 'OrderDate' in Orders df, three features are created to show 'OrderMonth', 'OrderWeekOfMonth' and 'OrderSeason'.

In [14]:
# rename the columns that can be also used in other datasets
Customers.rename(columns = {'City': 'CustomerCity', 'State': 'CustomerState'}, inplace=True)
# fill the missing values in Occupation and Industry columns with Unknown
Customers.fillna({'Occupation': 'Unknown', 'Industry': 'Unknown'}, inplace=True)
# drop the unnecessary columns for customer segmentation (all customers are from USA so country information is redundant)
Customers.drop(columns = ['Name', 'Email', 'Phone', 'Country', 'CreatedAt'], inplace = True)

# change the type of OrderDate column
Orders['OrderDate'] = pd.to_datetime(Orders['OrderDate'])

# create new features related to the order date 
Orders['OrderMonth'] = Orders['OrderDate'].dt.month
Orders['OrderWeekOfMonth'] = (Orders['OrderDate'].dt.day - 1) // 7 + 1
Orders['OrderSeason'] = Orders['OrderMonth'].apply(find_season)


CustomerBuys = Customers.merge(Orders, 
                               how = 'left',
                               on = 'CustomerID')
CustomerBuys.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5083 entries, 0 to 5082
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CustomerID        5083 non-null   int64         
 1   RetailerID        5083 non-null   int64         
 2   CustomerCity      5083 non-null   object        
 3   CustomerState     5083 non-null   object        
 4   Gender            5083 non-null   object        
 5   Age               5083 non-null   object        
 6   EmploymentStatus  5083 non-null   object        
 7   Education         5083 non-null   object        
 8   Occupation        5083 non-null   object        
 9   Industry          5083 non-null   object        
 10  Income            5083 non-null   int64         
 11  OrderID           1000 non-null   float64       
 12  OrderDate         1000 non-null   datetime64[ns]
 13  Count             1000 non-null   float64       
 14  TotalAmount       1000 n

In [15]:
customerbuys_na = CustomerBuys.isna().sum()
customerbuys_na[customerbuys_na > 0]

OrderID             4083
OrderDate           4083
Count               4083
TotalAmount         4083
Status              4083
OrderMonth          4083
OrderWeekOfMonth    4083
OrderSeason         4083
dtype: int64

Not all customers buy something, so when merging Customers with Orders we have missing information for each customer without any purchase. To handle missing values, the categorical column 'Status' will be filled with Unknown string and the numerical columns will be filled with 0 for missing values. However, OrderID and OrderDate will not be filled since they will be used in later steps.

In [17]:
CustomerBuys.fillna({'Status': 'Unknown'}, inplace = True)
CustomerBuys.fillna({'Count': 0, 'TotalAmount': 0, 'OrderMonth': 0, 'OrderWeekOfMonth': 0, 'OrderSeason': 0}, inplace=True)

In [18]:
customerbuys_na = CustomerBuys.isna().sum()
customerbuys_na[customerbuys_na > 0]

OrderID      4083
OrderDate    4083
dtype: int64

In [19]:
# check if there is any duplication
CustomerBuys[CustomerBuys.duplicated()].shape

(0, 19)

Since we have OrderID in CustomerBuys after merging with Orders, we can now merge the main df with OrderItems which give information about ProductID, price and quantity. OrderItemID is not a key since that column is not used by any other dataset. It is only used in OrderItems as index.

In [21]:
OrderItems.head(3)

,OrderItemID,OrderID,ProductID,Price,Quantity
0,1,1,6555,35.99,2
1,2,1,2673,11.99,2
2,3,1,3883,10.79,2


In [22]:
OrderItems = OrderItems.drop(columns = ['OrderItemID']).rename(columns = {'Price': 'ItemPrice'})

In [23]:
# use left join to merge OrderItems with CustomerBuys since CustomerBuys is our main dataset with customer information
CustomerBuys = CustomerBuys.merge(OrderItems, 
                                  how = 'left',
                                  on = 'OrderID')
CustomerBuys.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6078 entries, 0 to 6077
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CustomerID        6078 non-null   int64         
 1   RetailerID        6078 non-null   int64         
 2   CustomerCity      6078 non-null   object        
 3   CustomerState     6078 non-null   object        
 4   Gender            6078 non-null   object        
 5   Age               6078 non-null   object        
 6   EmploymentStatus  6078 non-null   object        
 7   Education         6078 non-null   object        
 8   Occupation        6078 non-null   object        
 9   Industry          6078 non-null   object        
 10  Income            6078 non-null   int64         
 11  OrderID           1995 non-null   float64       
 12  OrderDate         1995 non-null   datetime64[ns]
 13  Count             6078 non-null   float64       
 14  TotalAmount       6078 n

In [24]:
# show the columns with missing values
customerbuys_na = CustomerBuys.isna().sum()
customerbuys_na[customerbuys_na > 0]

OrderID      4083
OrderDate    4083
ProductID    4083
ItemPrice    4159
Quantity     4083
dtype: int64

In [25]:
# check if there is any duplication
CustomerBuys[CustomerBuys.duplicated()].shape

(0, 22)

In [26]:
CustomerBuys.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6078 entries, 0 to 6077
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CustomerID        6078 non-null   int64         
 1   RetailerID        6078 non-null   int64         
 2   CustomerCity      6078 non-null   object        
 3   CustomerState     6078 non-null   object        
 4   Gender            6078 non-null   object        
 5   Age               6078 non-null   object        
 6   EmploymentStatus  6078 non-null   object        
 7   Education         6078 non-null   object        
 8   Occupation        6078 non-null   object        
 9   Industry          6078 non-null   object        
 10  Income            6078 non-null   int64         
 11  OrderID           1995 non-null   float64       
 12  OrderDate         1995 non-null   datetime64[ns]
 13  Count             6078 non-null   float64       
 14  TotalAmount       6078 n

In CustomerBuys, we have now ProductID so we can merge Products dataset to learn each product's detail such as its main category.

In [28]:
Products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ProductID     10000 non-null  int64  
 1   RetailerID    10000 non-null  int64  
 2   Name          10000 non-null  object 
 3   MainCategory  10000 non-null  object 
 4   SubCategory   10000 non-null  object 
 5   Price         9605 non-null   float64
 6   SKU           10000 non-null  object 
 7   CreatedAt     10000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 625.1+ KB


While merging datasets, some key columns (ID columns) might get null values since some customers didn't purchase anything. If an ID column has a missing value, it automatically change its type to float while it was originally integer. To prevent error while merging, we change key columns from integer to float for merging purposes.

In [30]:
# drop unnecessary columns
# rename some columns for preventing confusion
Products = Products.drop(columns = ['CreatedAt', 'SKU']).rename(columns = {'Name': 'ProductName', 'Price': 'ProductPrice'})
Products['ProductID'] = Products['ProductID'].astype(float)
Products.drop(columns = ['RetailerID'], inplace = True)

CustomerBuys = CustomerBuys.merge(Products,
                                  how = 'left',
                                  on = ['ProductID'])

In [31]:
CustomerBuys.head()

,CustomerID,RetailerID,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,Industry,...,OrderMonth,OrderWeekOfMonth,OrderSeason,ProductID,ItemPrice,Quantity,ProductName,MainCategory,SubCategory,ProductPrice
0,1,48,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,11,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,manufacturing,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,66,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,24,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,other_services,...,4.0,1.0,2.0,5419.0,27.59,3.0,REMAXX Mini Portable Desktop Vacuum Cleaner Co...,appliances,All Appliances,27.59
4,5,38,San Diego city,California,male,65 to 74 years,employed,less_than_high_school_diploma,sales_and_office_occupations,wholesale_trade,...,1.0,2.0,1.0,1440.0,23.99,4.0,Jewels Galaxy Jewellery For Women Stackable Ri...,stores,Amazon Fashion,23.99


Most of the customers didn't buy anything but we still want to keep them in the dataset and clustering to understand their behavior and create targeted marketings. However, we create a purchase flag to distinguish them from the customers who purchased.

In [33]:
FlagDf = CustomerBuys.groupby('CustomerID')['TotalAmount'].sum().reset_index()
FlagDf.rename(columns = {'TotalAmount': 'PurchaseFlag'}, inplace = True)

# create a flag that if a customer purchased anything, it will give 1
FlagDf['PurchaseFlag'] = FlagDf['PurchaseFlag'].apply(lambda x: 1 if x > 0 else 0)

CustomerBuys = CustomerBuys.merge(FlagDf, 
                                  on = 'CustomerID', 
                                  how = 'left')
CustomerBuys.head(3)

,CustomerID,RetailerID,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,Industry,...,OrderWeekOfMonth,OrderSeason,ProductID,ItemPrice,Quantity,ProductName,MainCategory,SubCategory,ProductPrice,PurchaseFlag
0,1,48,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,11,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,manufacturing,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3,66,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


If a customer didn't purchase anything, their product information such as the columns given below should be equal to zero.

In [35]:
CustomerBuys.loc[CustomerBuys['PurchaseFlag'] == 0, ['Count', 'TotalAmount', 'Price', 'Quantity', 'TotalPrice']] = 0
CustomerBuys.head(3)

,CustomerID,RetailerID,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,Industry,...,ProductID,ItemPrice,Quantity,ProductName,MainCategory,SubCategory,ProductPrice,PurchaseFlag,Price,TotalPrice
0,1,48,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0.0,0.0
1,2,11,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,manufacturing,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0.0,0.0
2,3,66,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0.0,0.0


BehavioralData is not used because of the below example. In the main dataset (CustomerBuys), we don't have any of the CustomerID&ProductID combination shown in BehavioralData dataset. Even the observations with 'ActionType' = 'purchase' are not shown in CustomerBuys.

In [37]:
BehavioralData

,BehaviorID,CustomerID,ProductID,Timestamp,ClickSource,PageModule,ActionType,DwellTimeSeconds,DwellTimeCategory
0,1,3993,1969,2024-07-29 05:18:35.375653,sale,NaN,view,236,t5
1,2,758,4970,2024-07-11 21:22:35.375653,home,NaN,purchase,42,t3
2,3,215,7035,2024-07-29 19:08:35.375653,cart,specification,purchase,260,t5
3,4,2047,8433,2024-07-22 07:05:35.375653,category,specification,purchase,239,t5
4,5,4785,7389,2024-07-14 11:14:35.375653,home,comments,purchase,143,t5
...,...,...,...,...,...,...,...,...,...
9995,9996,3558,4142,2024-07-18 10:03:35.467440,cart,comments,add_to_cart,192,t5
9996,9997,1646,854,2024-07-24 05:44:35.467440,sale,NaN,add_to_cart,72,t4
9997,9998,4866,1370,2024-07-13 18:14:35.467440,search,bottom,add_to_cart,244,t5
9998,9999,2737,3781,2024-07-12 03:52:35.467440,sale,bottom,add_to_cart,44,t3


In [38]:
BehavioralData[BehavioralData['CustomerID'] == 4456.0]

,BehaviorID,CustomerID,ProductID,Timestamp,ClickSource,PageModule,ActionType,DwellTimeSeconds,DwellTimeCategory
456,457,4456,129,2024-08-07 01:06:35.378677,home,specification,view,211,t5
621,622,4456,9959,2024-08-04 12:02:35.380672,sale,bottom,purchase,219,t5
2095,2096,4456,8899,2024-08-06 16:50:35.399594,search,NaN,view,159,t5
3830,3831,4456,2010,2024-07-14 23:48:35.414554,cart,specification,view,70,t4
5460,5461,4456,4591,2024-07-23 06:13:35.429539,cart,comments,add_to_cart,54,t3
6303,6304,4456,272,2024-07-28 05:53:35.436521,home,bottom,add_to_cart,233,t5
6478,6479,4456,9045,2024-07-25 23:37:35.438515,category,NaN,purchase,197,t5
6675,6676,4456,4457,2024-08-05 16:15:35.439513,search,comments,purchase,284,t5
7271,7272,4456,9936,2024-07-25 16:29:35.444500,search,comments,purchase,256,t5


In [39]:
CustomerBuys[CustomerBuys['CustomerID'] == 4456.0][['CustomerID', 'ProductID']]

,CustomerID,ProductID
5393,4456,9888.0
5394,4456,3139.0


# SELLERS

As mentioned earlier, store and seller information will be gathered under SellerSells df then it will be joined with CustomerBuys at the last step.

Some dataframes keep only key identifiers and they can only be used for merging dataframes
- **ChannelsCampaigns:** StoreID, ChannelID
- **StoresCampaigns:** StoreID, CampaignID
- **AudiencesCampaigns:** AudienceID, CampaignID

In [43]:
Stores.head(3)

,StoreID,RetailerID,Name,City,State,Country,CreatedAt
0,1,57,Store 57,Los Angeles city,California,USA,2024-08-12 16:44:27.824364
1,2,15,Store 15,Austin city,Texas,USA,2024-08-12 16:44:27.825362
2,3,20,Store 20,Baltimore city,Maryland,USA,2024-08-12 16:44:27.825362


In [44]:
Stores.rename(columns = {'Name': 'StoreName', 'City': 'StoreCity', 'State': 'StoreState'}, inplace=True)
Stores.drop(columns = ['Country', 'CreatedAt'], inplace = True) # only USA

SellerSells = Stores.merge(ChannelsCampaigns, 
                           how = 'left', 
                           on = 'StoreID').merge(StoresCampaigns, 
                                                 how = 'left', 
                                                 on = 'StoreID')
SellerSells.head(3)

,StoreID,RetailerID,StoreName,StoreCity,StoreState,channelID,CampaignID
0,1,57,Store 57,Los Angeles city,California,42.0,60.0
1,1,57,Store 57,Los Angeles city,California,33.0,60.0
2,1,57,Store 57,Los Angeles city,California,79.0,60.0


By adding the keys to Stores and creating SellerSells, we can now use the information in Retailers and Channels.

However, Retailers dataframe doesn't give us information about customer segmentation. (RetailerID, Name, ContactInfo, CreatedAt, UserID). 

We can only use this dataframe to use UserID as a key to merge with Users dataframe but Users also doesn't have any necessary information (UserID, Username, Email, PasswordHash, CreatedAt).

Therefore, we will only merge Channels.

In [46]:
Channels.rename(columns = {'name': 'ChannelName'}, inplace = True)
SellerSells = SellerSells.merge(Channels.drop(columns = 'createdAt'), 
                                how = 'left', 
                                on = 'channelID')
SellerSells.head(3)

,StoreID,RetailerID,StoreName,StoreCity,StoreState,channelID,CampaignID,ChannelName,type,level
0,1,57,Store 57,Los Angeles city,California,42.0,60.0,Channel P1,email,one level
1,1,57,Store 57,Los Angeles city,California,33.0,60.0,Channel G1,social media,three level
2,1,57,Store 57,Los Angeles city,California,79.0,60.0,Channel A3,seo,three level


In [47]:
SellerSells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325 entries, 0 to 324
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StoreID      325 non-null    int64  
 1   RetailerID   325 non-null    int64  
 2   StoreName    325 non-null    object 
 3   StoreCity    325 non-null    object 
 4   StoreState   325 non-null    object 
 5   channelID    283 non-null    float64
 6   CampaignID   294 non-null    float64
 7   ChannelName  283 non-null    object 
 8   type         283 non-null    object 
 9   level        283 non-null    object 
dtypes: float64(2), int64(2), object(6)
memory usage: 27.9+ KB


Since we merged StoresCampaigns with Stores before, we now have CampaignID in SellerSells, which made it available for us to merge the dataset with MarketingCampaigns by using 'CampaignID'.

In [49]:
MarketingCampaigns.head(3)

,CampaignID,Name,StartDate,EndDate,Budget,CreatedAt
0,1,Campaign A,2024-08-08 21:50:33.556668,2024-09-07 21:50:33.557010,7626.44,2024-08-08 21:50:33.558220
1,2,Campaign B,2024-08-07 21:50:33.556719,2024-09-06 21:50:33.557013,3690.77,2024-08-07 21:50:33.558227
2,3,Campaign C,2024-08-06 21:50:33.556762,2024-09-05 21:50:33.557017,5664.16,2024-08-06 21:50:33.558232


In [50]:
MarketingCampaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CampaignID  100 non-null    int64  
 1   Name        100 non-null    object 
 2   StartDate   100 non-null    object 
 3   EndDate     100 non-null    object 
 4   Budget      100 non-null    float64
 5   CreatedAt   100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


StartDate and EndDate are object-typed but we can use them for feature engineering after changing their type to datetime.

In [52]:
MarketingCampaigns['StartDate'] = pd.to_datetime(MarketingCampaigns['StartDate'])
MarketingCampaigns['EndDate'] = pd.to_datetime(MarketingCampaigns['EndDate'])
MarketingCampaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CampaignID  100 non-null    int64         
 1   Name        100 non-null    object        
 2   StartDate   100 non-null    datetime64[ns]
 3   EndDate     100 non-null    datetime64[ns]
 4   Budget      100 non-null    float64       
 5   CreatedAt   100 non-null    object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 4.8+ KB


In [53]:
# All campaigns start and end at 21:50, so the time is redundant
MarketingCampaigns['StartTime'] = MarketingCampaigns['StartDate'].dt.time

# Check
MarketingCampaigns['StartTime'].sort_values() # ascending

0     21:50:33.556668
1     21:50:33.556719
2     21:50:33.556762
3     21:50:33.556768
4     21:50:33.556771
           ...       
95    21:50:33.556995
96    21:50:33.556997
97    21:50:33.556999
98    21:50:33.557002
99    21:50:33.557004
Name: StartTime, Length: 100, dtype: object

In [54]:
# keep only dates and remove times from StartDate and EndDate
MarketingCampaigns['StartDate'] = MarketingCampaigns['StartDate'].dt.date
MarketingCampaigns['EndDate'] = MarketingCampaigns['EndDate'].dt.date

# Reconvert to datetime
MarketingCampaigns['StartDate'] = pd.to_datetime(MarketingCampaigns['StartDate'])
MarketingCampaigns['EndDate'] = pd.to_datetime(MarketingCampaigns['EndDate'])

# Estimate how many days a campaign takes
MarketingCampaigns['CampaignDuration'] = (MarketingCampaigns['EndDate'] - MarketingCampaigns['StartDate']).dt.days
MarketingCampaigns.head(3)

,CampaignID,Name,StartDate,EndDate,Budget,CreatedAt,StartTime,CampaignDuration
0,1,Campaign A,2024-08-08,2024-09-07,7626.44,2024-08-08 21:50:33.558220,21:50:33.556668,30
1,2,Campaign B,2024-08-07,2024-09-06,3690.77,2024-08-07 21:50:33.558227,21:50:33.556719,30
2,3,Campaign C,2024-08-06,2024-09-05,5664.16,2024-08-06 21:50:33.558232,21:50:33.556762,30


In [55]:
# All campaigns take 30 days so the duration is redundant
MarketingCampaigns[MarketingCampaigns['CampaignDuration'] != 30]

,CampaignID,Name,StartDate,EndDate,Budget,CreatedAt,StartTime,CampaignDuration


In [56]:
# remove unnecessary columns
MarketingCampaigns = MarketingCampaigns.drop(columns = ['CreatedAt', 'StartTime', 'CampaignDuration', 'Budget'])
MarketingCampaigns['CampaignID'] = MarketingCampaigns['CampaignID'].astype(float)
MarketingCampaigns.rename(columns = {'Name': 'CampaignName'}, inplace = True)
MarketingCampaigns.head(3)

,CampaignID,CampaignName,StartDate,EndDate
0,1.0,Campaign A,2024-08-08,2024-09-07
1,2.0,Campaign B,2024-08-07,2024-09-06
2,3.0,Campaign C,2024-08-06,2024-09-05


In [57]:
SellerSells.head(3)

,StoreID,RetailerID,StoreName,StoreCity,StoreState,channelID,CampaignID,ChannelName,type,level
0,1,57,Store 57,Los Angeles city,California,42.0,60.0,Channel P1,email,one level
1,1,57,Store 57,Los Angeles city,California,33.0,60.0,Channel G1,social media,three level
2,1,57,Store 57,Los Angeles city,California,79.0,60.0,Channel A3,seo,three level


In [58]:
SellerSells = SellerSells.merge(MarketingCampaigns, 
                                how = 'left',
                                on = 'CampaignID')
SellerSells.head(3)

,StoreID,RetailerID,StoreName,StoreCity,StoreState,channelID,CampaignID,ChannelName,type,level,CampaignName,StartDate,EndDate
0,1,57,Store 57,Los Angeles city,California,42.0,60.0,Channel P1,email,one level,Campaign H,2024-06-10,2024-07-10
1,1,57,Store 57,Los Angeles city,California,33.0,60.0,Channel G1,social media,three level,Campaign H,2024-06-10,2024-07-10
2,1,57,Store 57,Los Angeles city,California,79.0,60.0,Channel A3,seo,three level,Campaign H,2024-06-10,2024-07-10


In [59]:
CustomerBuys.head(3)

,CustomerID,RetailerID,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,Industry,...,ProductID,ItemPrice,Quantity,ProductName,MainCategory,SubCategory,ProductPrice,PurchaseFlag,Price,TotalPrice
0,1,48,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0.0,0.0
1,2,11,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,manufacturing,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0.0,0.0
2,3,66,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,0.0,0.0


In [60]:
# create main df by merging customer and seller information
df = CustomerBuys.merge(SellerSells, 
                        how = 'left', 
                        on = 'RetailerID')
df.head()

,CustomerID,RetailerID,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,Industry,...,StoreCity,StoreState,channelID,CampaignID,ChannelName,type,level,CampaignName,StartDate,EndDate
0,1,48,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,2,11,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,manufacturing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,3,66,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,4,24,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,other_services,...,Louisville/Jefferson County metro government (...,Kentucky,67.0,17.0,Channel O2,podcast,three level,Campaign Q,2024-07-23,2024-08-22
4,4,24,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,other_services,...,Louisville/Jefferson County metro government (...,Kentucky,67.0,60.0,Channel O2,podcast,three level,Campaign H,2024-06-10,2024-07-10


We will add a new feature to the dataframe to show if the customer make their purchase during a campaign period.

In [62]:
# customer with CampaignFlag=1 purchased their items during the campaign period
df['CampaignFlag'] = ((df['OrderDate'] >= df['StartDate']) & (df['OrderDate'] <= df['EndDate'])).astype(int)
df[df['CampaignFlag']==1].head(3)

,CustomerID,RetailerID,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,Industry,...,StoreState,channelID,CampaignID,ChannelName,type,level,CampaignName,StartDate,EndDate,CampaignFlag
139,23,90,Tucson city,Arizona,female,45 to 54 years,employed,high_school_graduate,management_business_science_arts,educational_services_health_care_social_assist...,...,Ohio,19.0,43.0,Channel S,influencer,three level,Campaign Q,2024-06-27,2024-07-27,1
140,23,90,Tucson city,Arizona,female,45 to 54 years,employed,high_school_graduate,management_business_science_arts,educational_services_health_care_social_assist...,...,Ohio,19.0,57.0,Channel S,influencer,three level,Campaign E,2024-06-13,2024-07-13,1
142,23,90,Tucson city,Arizona,female,45 to 54 years,employed,high_school_graduate,management_business_science_arts,educational_services_health_care_social_assist...,...,Ohio,67.0,43.0,Channel O2,podcast,three level,Campaign Q,2024-06-27,2024-07-27,1


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22421 entries, 0 to 22420
Data columns (total 42 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CustomerID        22421 non-null  int64         
 1   RetailerID        22421 non-null  int64         
 2   CustomerCity      22421 non-null  object        
 3   CustomerState     22421 non-null  object        
 4   Gender            22421 non-null  object        
 5   Age               22421 non-null  object        
 6   EmploymentStatus  22421 non-null  object        
 7   Education         22421 non-null  object        
 8   Occupation        22421 non-null  object        
 9   Industry          22421 non-null  object        
 10  Income            22421 non-null  int64         
 11  OrderID           7395 non-null   float64       
 12  OrderDate         7395 non-null   datetime64[ns]
 13  Count             22421 non-null  float64       
 14  TotalAmount       2242

In [64]:
# remove all the redundant columns for customer segmentation
df.drop(columns = ['RetailerID', 'OrderDate', 'Status', 'ProductID', 'ProductName', 'Price', 'TotalPrice', 'StoreID', 'StoreName', 
                   'StoreCity', 'StoreState', 'channelID', 'CampaignID', 'ChannelName', 'level', 'CampaignName', 'StartDate', 
                   'EndDate'], inplace = True)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22421 entries, 0 to 22420
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        22421 non-null  int64  
 1   CustomerCity      22421 non-null  object 
 2   CustomerState     22421 non-null  object 
 3   Gender            22421 non-null  object 
 4   Age               22421 non-null  object 
 5   EmploymentStatus  22421 non-null  object 
 6   Education         22421 non-null  object 
 7   Occupation        22421 non-null  object 
 8   Industry          22421 non-null  object 
 9   Income            22421 non-null  int64  
 10  OrderID           7395 non-null   float64
 11  Count             22421 non-null  float64
 12  TotalAmount       22421 non-null  float64
 13  OrderMonth        22421 non-null  float64
 14  OrderWeekOfMonth  22421 non-null  float64
 15  OrderSeason       22421 non-null  float64
 16  ItemPrice         7056 non-null   float6

We aggregate some of the remaining columns to have only one row for each customer. For example, the most purchased main category is kept for each customer.

In [67]:
aggregated_df = df.groupby('CustomerID').agg(
    CustomerCity = ('CustomerCity', 'first'),
    CustomerState = ('CustomerState', 'first'),
    Gender = ('Gender', 'first'),
    Age = ('Age', 'first'),
    EmploymentStatus = ('EmploymentStatus', 'first'),
    Education = ('Education', 'first'),
    Occupation = ('Occupation', 'first'),
    Industry = ('Industry', 'first'),
    Income = ('Income', 'first'),
    OrderCount = ('OrderID', 'count'),
    OrderItemMean = ('Count', 'mean'),
    TotalAmountMax = ('TotalAmount', 'max'),
    TotalAmountMin = ('TotalAmount', 'min'),
    OrderMonthMode = ('OrderMonth', lambda x: x.mode()[0] if not x.mode().empty else np.nan),
    OrderWeekOfMonthMode = ('OrderWeekOfMonth', lambda x: x.mode()[0] if not x.mode().empty else np.nan),
    OrderSeasonMode = ('OrderSeason', lambda x: x.mode()[0] if not x.mode().empty else np.nan),
    ItemPriceMean = ('ItemPrice', 'mean'),
    MainCategoryMode = ('MainCategory', lambda x: x.mode()[0] if not x.mode().empty else np.nan),
    SubCategoryMode = ('SubCategory', lambda x: x.mode()[0] if not x.mode().empty else np.nan),
    TypeMode = ('type', lambda x: x.mode()[0] if not x.mode().empty else np.nan),
    PurchaseFlag = ('PurchaseFlag', 'first'),
    CampaignFlag = ('CampaignFlag', lambda x: 1 if (x == 1).any() else 0)
).reset_index()

aggregated_df.head()

,CustomerID,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,Industry,Income,...,TotalAmountMin,OrderMonthMode,OrderWeekOfMonthMode,OrderSeasonMode,ItemPriceMean,MainCategoryMode,SubCategoryMode,TypeMode,PurchaseFlag,CampaignFlag
0,1,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,46148,...,0.00,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0,0
1,2,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,manufacturing,42639,...,0.00,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0,0
2,3,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,Unknown,Unknown,42019,...,0.00,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0,0
3,4,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,other_services,49461,...,82.77,4.0,1.0,2.0,27.59,appliances,All Appliances,podcast,1,0
4,5,San Diego city,California,male,65 to 74 years,employed,less_than_high_school_diploma,sales_and_office_occupations,wholesale_trade,60797,...,95.96,1.0,2.0,1.0,23.99,stores,Amazon Fashion,NaN,1,0


In [68]:
# check the missing values
missingness = aggregated_df.isna().sum()
missingness[missingness > 0]

ItemPriceMean       4096
MainCategoryMode    4083
SubCategoryMode     4083
TypeMode            2620
dtype: int64

In [69]:
# fill the missing values with 0 or Unknown for the customers who didn't purchase anything
aggregated_df.fillna({'ItemPriceMean': 0, 'MainCategoryMode': 'Unknown', 'SubCategoryMode': 'Unknown', 'TypeMode': 'Unknown'}, inplace = True)

In [70]:
# double check the missing values
missingness = aggregated_df.isna().sum()
missingness[missingness > 0]

Series([], dtype: int64)

In [71]:
aggregated_df.shape

(5000, 23)

In [72]:
path2 = '/Users/ilaydabekircan/Documents/Vision Bridge/DS Assignment/df_clean.csv'
aggregated_df.to_csv(path2, index = False)